In [1]:
#import packages for plotting

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



from Bandpass import butter_bandpass_filter
from Hilbert import Hilbert_transform,phase
from CoherenceMap import CoherenceMap

In [2]:
#read all datafiles

n = 215 #highest ID of face filename

notfound = {}

for i in range(n):
    try:
            df = pd.read_csv(f"AALdata/sub-{(i+1):04d}_faces.csv",sep=",",header=None)
    except FileNotFoundError:
            notfound[i] = f"{(i+1):04d}"
            continue
    
    # Filtering signal with bandpass filter
    X_filtered = butter_bandpass_filter(df.values)


    # Hilbert transform to extract phase data.
    N,T = X_filtered.shape
    Theta = np.zeros((N,T))
    for idx, x in enumerate(X_filtered):
        s_t,sh_t=Hilbert_transform(x)
        Theta_t = phase(s_t, sh_t)
        Theta[idx,:] = Theta_t


    # Creating coherance map and extracting leading eigenvector
    LEiDA_Signal = CoherenceMap(Theta)
    
    #Add the i'th subject  to dataframe 
    X = LEiDA_Signal if i == 0 else np.hstack((X,LEiDA_Signal))

print(f"files not found {[*notfound.values()]}")
print(f"Number of files not found: {len(notfound)}")
print(f"Dimentions of data ({X.shape[0],X.shape[1]})")

KeyboardInterrupt: 

In [3]:
import pickle
with open('.Data_LEiDA_Representation.pickle','wb') as f:
    pickle.dump(X, f)